In [13]:
import numpy as np
import pandas as pd
import os
from itertools import product # Used for generating 'L'/'H' combinations for N-D sub-bands


In [14]:
def haar_lwt_1d_decompose(data):
    """
    Performs a single level 1D Haar Lifting Wavelet Transform decomposition.

    Args:
        data (np.ndarray): The 1D input data array.

    Returns:
        tuple: A tuple containing:
            - approximation (np.ndarray): The approximation coefficients.
            - detail (np.ndarray): The detail coefficients.
            - original_len (int): The original length of the input data before padding.
    """
    original_len = len(data)
    
    # Pad if length is odd to ensure even split
    if original_len % 2 != 0:
        padded_data = np.pad(data, (0, 1), 'constant', constant_values=0)
    else:
        padded_data = data

    # Split: Separate into even and odd indexed samples
    even = padded_data[::2]
    odd = padded_data[1::2]

    # Predict: Calculate detail coefficients (d_j = odd - even)
    detail = odd - even

    # Update: Calculate approximation coefficients (s_j = even + d_j / 2)
    approximation = even + detail / 2

    return approximation, detail, original_len

def haar_lwt_1d_reconstruct(approximation, detail, original_len):
    """
    Reconstructs a 1D signal from its single level Haar Lifting Wavelet Transform coefficients.

    Args:
        approximation (np.ndarray): The approximation coefficients.
        detail (np.ndarray): The detail coefficients.
        original_len (int): The original length of the signal before decomposition padding.

    Returns:
        np.ndarray: The reconstructed 1D signal.
    """
    # Inverse Update
    even = approximation - detail / 2

    # Inverse Predict
    odd = detail + even

    # Merge: Interleave even and odd parts
    combined_padded_len = len(even) + len(odd)
    reconstructed = np.empty(combined_padded_len, dtype=float)
    reconstructed[::2] = even
    reconstructed[1::2] = odd
    
    # Trim to original length
    return reconstructed[:original_len]

def _apply_1d_lwt_along_axis(data_nd, axis):
    """
    Applies 1D Haar LWT decomposition along a specified axis of an N-dimensional array.

    Args:
        data_nd (np.ndarray): The N-dimensional input data array.
        axis (int): The axis along which to apply the 1D transform.

    Returns:
        tuple: A tuple containing:
            - approx_coeffs_nd (np.ndarray): N-dimensional approximation coefficients.
            - detail_coeffs_nd (np.ndarray): N-dimensional detail coefficients.
            - original_len_axis (int): The original length of the specified axis before padding.
    """
    original_shape = data_nd.shape
    original_len_axis = original_shape[axis]

    # Handle padding for N-D: If original_shape[axis] is odd, pad along that axis.
    pad_width = [(0, 0)] * data_nd.ndim
    if original_len_axis % 2 != 0:
        pad_width[axis] = (0, 1)
        padded_data_nd = np.pad(data_nd, pad_width, 'constant', constant_values=0)
    else:
        padded_data_nd = data_nd

    # Get even and odd slices along the specified axis
    slicer_even = [slice(None)] * padded_data_nd.ndim
    slicer_even[axis] = np.arange(0, padded_data_nd.shape[axis], 2)
    even_part = padded_data_nd[tuple(slicer_even)]

    slicer_odd = [slice(None)] * padded_data_nd.ndim
    slicer_odd[axis] = np.arange(1, padded_data_nd.shape[axis], 2)
    odd_part = padded_data_nd[tuple(slicer_odd)]

    # Predict: detail = odd - even
    detail_coeffs_nd = odd_part - even_part

    # Update: approximation = even + detail / 2
    approx_coeffs_nd = even_part + detail_coeffs_nd / 2

    return approx_coeffs_nd, detail_coeffs_nd, original_len_axis

def _apply_1d_inverse_lwt_along_axis(approx_coeffs_nd, detail_coeffs_nd, axis, original_len_axis):
    """
    Applies 1D Haar LWT reconstruction along a specified axis of N-dimensional approximation and detail coefficients.

    Args:
        approx_coeffs_nd (np.ndarray): N-dimensional approximation coefficients.
        detail_coeffs_nd (np.ndarray): N-dimensional detail coefficients.
        axis (int): The axis along which to apply the 1D inverse transform.
        original_len_axis (int): The original length of the axis before decomposition padding.

    Returns:
        np.ndarray: The reconstructed N-dimensional array.
    """
    # Inverse Update: even = approx - detail / 2
    even_part = approx_coeffs_nd - detail_coeffs_nd / 2

    # Inverse Predict: odd = detail + even
    odd_part = detail_coeffs_nd + even_part

    # Merge: Interleave even and odd parts
    # Determine the shape of the reconstructed array before trimming
    reconstructed_shape = list(even_part.shape)
    reconstructed_shape[axis] = even_part.shape[axis] + odd_part.shape[axis]
    reconstructed_padded_nd = np.empty(reconstructed_shape, dtype=float)

    # Place even parts at even indices and odd parts at odd indices along the axis
    slicer_even_out = [slice(None)] * reconstructed_padded_nd.ndim
    slicer_even_out[axis] = np.arange(0, reconstructed_padded_nd.shape[axis], 2)
    reconstructed_padded_nd[tuple(slicer_even_out)] = even_part

    slicer_odd_out = [slice(None)] * reconstructed_padded_nd.ndim
    slicer_odd_out[axis] = np.arange(1, reconstructed_padded_nd.shape[axis], 2)
    reconstructed_padded_nd[tuple(slicer_odd_out)] = odd_part

    # Trim to the original length along the specified axis
    trim_slicer = [slice(None)] * reconstructed_padded_nd.ndim
    trim_slicer[axis] = slice(0, original_len_axis)
    
    return reconstructed_padded_nd[tuple(trim_slicer)]

def haar_lwt_nd_decompose(data, level):
    """
    Performs an N-level N-dimensional Haar Lifting Wavelet Transform decomposition.

    Args:
        data (np.ndarray): The N-dimensional input data array.
        level (int): The number of decomposition levels to perform.

    Returns:
        dict: A dictionary containing:
              - 'original_shape': tuple, original shape of the input data.
              - 'final_approx': np.ndarray, the final approximation sub-band (all Ls).
              - 'level_X_details': dict, where X is the level number.
                - Keys are sub-band names (e.g., 'LH', 'HL', 'HH' for 2D, or 'LLH', 'LHL', etc. for 3D).
                  The name is a string of 'L' and 'H' characters, where 'L' means low-pass
                  and 'H' means high-pass along the corresponding dimension. The order of characters
                  corresponds to the order of dimensions (axis 0, axis 1, ...).
                - Values are np.ndarray, the coefficient arrays for that sub-band.
              - 'level_X_input_shapes_before_axis_transform': dict.
                - Keys are the prefixes (L/H combinations) of the sub-bands *before* a 1D transform
                  was applied along a specific axis.
                - Values are the full N-dimensional shape of that sub-band *before* the 1D transform.
                  This is crucial for reconstruction.
    """
    current_approx = np.array(data, dtype=float)
    coeffs_tree = {'original_shape': data.shape}
    
    ndim = data.ndim

    for l in range(1, level + 1):
        details_this_level = {}
        # Stores the shape of the array that was input to the 1D transform along each axis.
        # Key: The L/H prefix of the sub-band *before* the current axis was processed.
        # Value: The shape of that sub-band.
        level_l_input_shapes_before_axis_transform = {}

        current_set_of_arrays = {'': current_approx} # Start with empty prefix for the initial array
        
        # Iterate through each dimension (axis)
        for dim_idx in range(ndim):
            next_set_of_arrays = {}
            
            # Iterate through the arrays accumulated from previous dimension transforms
            for input_prefix, arr_to_process in current_set_of_arrays.items():
                if arr_to_process.size == 0: # Skip empty arrays if they resulted from previous padding
                    continue

                # Store the shape of the array *before* applying 1D LWT along this dim_idx
                # The key here is the prefix *leading up to* this dimension.
                # This shape is needed for reconstruction.
                level_l_input_shapes_before_axis_transform[input_prefix] = arr_to_process.shape

                # Apply 1D LWT along the current dimension
                approx_part, detail_part, _ = _apply_1d_lwt_along_axis(arr_to_process, dim_idx) # _ is original_len_axis
                
                # Store the approximation and detail parts with updated prefixes
                next_set_of_arrays[input_prefix + 'L'] = approx_part
                next_set_of_arrays[input_prefix + 'H'] = detail_part
            
            # Update the set of arrays for the next dimension's processing
            current_set_of_arrays = next_set_of_arrays
        
        # After processing all dimensions for this level, `current_set_of_arrays`
        # contains all 2^ndim sub-bands.
        # The 'L'*ndim key holds the approximation for the next level.
        current_approx = current_set_of_arrays.pop('L' * ndim)
        
        # The remaining items in `current_set_of_arrays` are the detail sub-bands for this level.
        details_this_level = current_set_of_arrays

        coeffs_tree[f'level_{l}_details'] = details_this_level
        coeffs_tree[f'level_{l}_input_shapes_before_axis_transform'] = level_l_input_shapes_before_axis_transform
        
        # Check if the approximation sub-band is too small for further decomposition
        if any(s < 2 for s in current_approx.shape) or current_approx.size == 0:
            print(f"Warning: Stopped N-D decomposition at level {l} because approximation sub-band became too small: {current_approx.shape}")
            break
    
    coeffs_tree['final_approx'] = current_approx
    return coeffs_tree

def haar_lwt_nd_reconstruct(coeffs_tree):
    """
    Reconstructs an N-dimensional signal from its N-level Haar Lifting Wavelet Transform coefficients.

    Args:
        coeffs_tree (dict): A dictionary containing the final approximation sub-band
                            and all detail sub-bands (LH, HL, HH, etc.) for each level,
                            as returned by `haar_lwt_nd_decompose`.

    Returns:
        np.ndarray: The reconstructed N-dimensional signal.
    """
    current_reconstruction = coeffs_tree['final_approx']
    original_full_shape = coeffs_tree['original_shape']
    ndim = len(original_full_shape)

    # Determine the number of levels from the keys in coeffs_tree
    levels = 0
    for key in coeffs_tree:
        if key.startswith('level_') and key.endswith('_details'):
            levels = max(levels, int(key.split('_')[1]))
    
    # Reconstruct level by level, from coarsest to finest
    for l in range(levels, 0, -1):
        details_this_level = coeffs_tree[f'level_{l}_details']
        input_shapes_before_axis_transform = coeffs_tree[f'level_{l}_input_shapes_before_axis_transform']

        # Start with all 2^ndim sub-bands for this level, including the current_reconstruction (LL...L)
        all_sub_bands_at_this_level = details_this_level.copy()
        all_sub_bands_at_this_level['L' * ndim] = current_reconstruction

        # Iterate through dimensions in reverse order for reconstruction
        for dim_idx in range(ndim - 1, -1, -1):
            next_reconstructed_arrays = {}
            
            # Generate all possible prefixes for the dimensions *before* dim_idx (L/H combinations)
            prefix_combinations = [''.join(p) for p in product('LH', repeat=dim_idx)]
            # Suffixes for dimensions *after* dim_idx, which have already been reconstructed ('R' combinations)
            reconstruct_suffix_combinations = [''.join(p) for p in product('R', repeat=(ndim - 1 - dim_idx))]

            for p_before in prefix_combinations:
                for r_suffix in reconstruct_suffix_combinations:
                    approx_key = p_before + 'L' + r_suffix
                    detail_key = p_before + 'H' + r_suffix
                    
                    if approx_key in all_sub_bands_at_this_level and detail_key in all_sub_bands_at_this_level:
                        approx_part = all_sub_bands_at_this_level[approx_key]
                        detail_part = all_sub_bands_at_this_level[detail_key]
                        
                        # The prefix used when this `dim_idx` was processed in the forward pass
                        # is simply `p_before`.
                        input_prefix_for_lookup = p_before
                        
                        original_input_shape_for_this_dim = input_shapes_before_axis_transform.get(input_prefix_for_lookup)

                        if original_input_shape_for_this_dim is None:
                            raise KeyError(f"Original input shape not found for (dim_idx={dim_idx}, input_prefix='{input_prefix_for_lookup}') at level {l}. "
                                           f"Approx key: '{approx_key}', Detail key: '{detail_key}'")

                        original_len_for_this_axis = original_input_shape_for_this_dim[dim_idx]

                        reconstructed_part = _apply_1d_inverse_lwt_along_axis(
                            approx_part, detail_part, dim_idx, original_len_for_this_axis
                        )
                        
                        # The key for the next level of reconstruction is `p_before + 'R' + r_suffix`
                        # This key needs to be consistent for the next iteration of dim_idx.
                        # The 'R' for the current dim_idx is added here.
                        next_reconstructed_arrays[p_before + 'R' + r_suffix] = reconstructed_part
            
            # CRITICAL FIX: Update all_sub_bands_at_this_level for the next dimension's processing
            all_sub_bands_at_this_level = next_reconstructed_arrays
        
        # After all dimensions are reconstructed for this level, there should be only one array left
        # which is the full approximation for the previous level.
        current_reconstruction = all_sub_bands_at_this_level['R' * ndim]
    
    # Final trim to the original input data shape
    final_reconstruction_slicer = tuple(slice(0, s) for s in original_full_shape)
    return current_reconstruction[final_reconstruction_slicer]


def save_coefficients_to_files(output_dir, coeffs_tree, filename='all_haar_lwt_coeffs_columns_nd.csv'):
    """
    Saves all Haar LWT coefficients (approximation and all detail levels) into a single .csv file,
    with each coefficient type in its own column. Handles N-dimensional data.

    Args:
        output_dir (str): The directory where the coefficient file will be saved.
                          If the directory does not exist, it will be created.
        coeffs_tree (dict): A dictionary containing the final approximation sub-band
                            and all detail sub-bands for each level,
                            as returned by `haar_lwt_nd_decompose`.
    """
    os.makedirs(output_dir, exist_ok=True)

    all_coeff_arrays_flat = []
    headers = []

    # Add the final approximation (LL...L)
    if 'final_approx' in coeffs_tree:
        all_coeff_arrays_flat.append(coeffs_tree['final_approx'].flatten())
        headers.append('Final_Approximation')

    # Determine the number of levels and dimensions
    levels = 0
    ndim = len(coeffs_tree['original_shape'])
    for key in coeffs_tree:
        if key.startswith('level_') and key.endswith('_details'):
            levels = max(levels, int(key.split('_')[1]))

    # Generate all possible L/H combinations for N dimensions
    all_lh_combinations = [''.join(p) for p in product('LH', repeat=ndim)]
    
    # Exclude the 'L'*ndim combination as it's the approximation for the next level
    # or the final_approx.
    detail_lh_combinations = [c for c in all_lh_combinations if c != 'L' * ndim]

    # Add detail coefficients for each level
    for l in range(1, levels + 1):
        details_this_level = coeffs_tree.get(f'level_{l}_details', {})
        # Sort detail combinations for consistent column order in CSV
        for combo in sorted(detail_lh_combinations):
            if combo in details_this_level and details_this_level[combo].size > 0:
                all_coeff_arrays_flat.append(details_this_level[combo].flatten())
                headers.append(f'{combo}_L{l}')

    # Find the maximum length among all flattened coefficient arrays
    max_len = 0
    if all_coeff_arrays_flat:
        max_len = max(len(arr) for arr in all_coeff_arrays_flat)

    # Pad shorter arrays with NaN
    padded_coeff_arrays = []
    for arr in all_coeff_arrays_flat:
        if len(arr) < max_len:
            padded_arr = np.pad(arr, (0, max_len - len(arr)), 'constant', constant_values=np.nan)
        else:
            padded_arr = arr
        padded_coeff_arrays.append(padded_arr)

    # Stack the padded arrays horizontally
    if padded_coeff_arrays:
        combined_coeffs_2d = np.column_stack(padded_coeff_arrays)
    else:
        combined_coeffs_2d = np.array([[]])

    header_str = ','.join(headers)

    combined_filepath = os.path.join(output_dir, filename)
    np.savetxt(combined_filepath, combined_coeffs_2d, delimiter=',', header=header_str, comments='')
    print(f"Saved all Haar LWT coefficients (in columns) to: {combined_filepath}")

# --- Example Usage ---
if __name__ == "__main__":

    df = pd.read_csv('Data_August_Renewable.csv')
    data = df['Speed'].values
    n_levels = 3
    print(f"Original Data: {data}")
    print(f"Decomposition Levels: {n_levels}")
    print("-" * 30) 
    coeffs = haar_lwt_nd_decompose(data, n_levels)
    print("\n--- Decomposition Results ---")
    print(f"Final LL (Approximation) Sub-band:\n{coeffs['final_approx']}")
    for l in range(1, n_levels + 1):
        details = coeffs.get(f'level_{l}_details', {})
        for key, val in details.items():
            print(f"  {key}_L{l} Detail:\n{val}")
    print("-" * 30)
    output_directory = './'
    save_coefficients_to_files(output_directory, coeffs, filename='haar_lwt_1D_L3_coeffs_speed_dataset.csv')
    print(f"Saved coefficients to directory: {output_directory}/haar_lwt_1D_L3_coeffs_speed_dataset.csv")
    print("-" * 30)
    reconstructed_data = haar_lwt_nd_reconstruct(coeffs)
    print("\n--- Reconstruction Results ---")
    print(f"Reconstructed Data: {reconstructed_data}")
    is_reconstruction_accurate = np.allclose(data, reconstructed_data)
    print(f"Is Reconstruction Accurate? {is_reconstruction_accurate}")
    if not is_reconstruction_accurate:
        print(f"Difference: {data - reconstructed_data}")
    print("-" * 30)

    # # --- 1D Example (still works with the new N-D functions) ---
    # print("--- 1D Example ---")
    # data_1d = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
    # n_levels_1d = 2

    # print(f"Original 1D Data: {data_1d}")
    # print(f"Decomposition Levels: {n_levels_1d}")
    # print("-" * 30)

    # coeffs_1d = haar_lwt_nd_decompose(data_1d, n_levels_1d)
    # print("\n--- 1D Decomposition Results ---")
    # print(f"Final LL (Approximation) Sub-band:\n{coeffs_1d['final_approx']}")
    # for l in range(1, n_levels_1d + 1):
    #     details = coeffs_1d.get(f'level_{l}_details', {})
    #     for key, val in details.items():
    #         print(f"  {key}_L{l} Detail:\n{val}")
    # print("-" * 30)

    # output_directory_1d = 'haar_lwt_coeffs_1d'
    # save_coefficients_to_files(output_directory_1d, coeffs_1d)
    # print("-" * 30)

    # reconstructed_data_1d = haar_lwt_nd_reconstruct(coeffs_1d)
    # print("\n--- 1D Reconstruction Results ---")
    # print(f"Reconstructed Data: {reconstructed_data_1d}")
    # is_reconstruction_accurate_1d = np.allclose(data_1d, reconstructed_data_1d)
    # print(f"Is 1D Reconstruction Accurate? {is_reconstruction_accurate_1d}")
    # if not is_reconstruction_accurate_1d:
    #     print(f"Difference: {data_1d - reconstructed_data_1d}")
    # print("-" * 30)


    # # --- 2D Example ---
    # print("\n--- 2D Example ---")
    # data_2d = np.array([
    #     [1, 2, 3, 4, 5],
    #     [6, 7, 8, 9, 10],
    #     [11, 12, 13, 14, 15],
    #     [16, 17, 18, 19, 20],
    #     [21, 22, 23, 24, 25]
    # ])
    # n_levels_2d = 2

    # print(f"Original 2D Data:\n{data_2d}")
    # print(f"Original 2D Data Shape: {data_2d.shape}")
    # print(f"Decomposition Levels: {n_levels_2d}")
    # print("-" * 30)

    # coeffs_2d = haar_lwt_nd_decompose(data_2d, n_levels_2d)

    # print("\n--- 2D Decomposition Results ---")
    # print(f"Final LL (Approximation) Sub-band:\n{coeffs_2d['final_approx']}")
    # for l in range(1, n_levels_2d + 1):
    #     details = coeffs_2d.get(f'level_{l}_details', {})
    #     # Sort keys for predictable output
    #     for key in sorted(details.keys()):
    #         val = details[key]
    #         print(f"  {key}_L{l} Detail:\n{val}")
    # print("-" * 30)

    # output_directory_2d = 'haar_lwt_coeffs_2d'
    # save_coefficients_to_files(output_directory_2d, coeffs_2d)
    # print("-" * 30)

    # reconstructed_data_2d = haar_lwt_nd_reconstruct(coeffs_2d)

    # print("\n--- 2D Reconstruction Results ---")
    # print(f"Reconstructed Data:\n{reconstructed_data_2d}")
    # is_reconstruction_accurate_2d = np.allclose(data_2d, reconstructed_data_2d)
    # print(f"Is 2D Reconstruction Accurate? {is_reconstruction_accurate_2d}")
    # if not is_reconstruction_accurate_2d:
    #     print(f"Difference:\n{data_2d - reconstructed_data_2d}")
    # print("-" * 30)

    # # --- 3D Example ---
    # print("\n--- 3D Example ---")
    # data_3d = np.arange(1, 28).reshape((3, 3, 3)) # A 3x3x3 array
    # n_levels_3d = 1 # For 3x3x3, only 1 level is practical as dimensions become 2x2x2 after 1 level

    # print(f"Original 3D Data:\n{data_3d}")
    # print(f"Original 3D Data Shape: {data_3d.shape}")
    # print(f"Decomposition Levels: {n_levels_3d}")
    # print("-" * 30)

    # coeffs_3d = haar_lwt_nd_decompose(data_3d, n_levels_3d)

    # print("\n--- 3D Decomposition Results ---")
    # print(f"Final LLL (Approximation) Sub-band:\n{coeffs_3d['final_approx']}")
    # for l in range(1, n_levels_3d + 1):
    #     details = coeffs_3d.get(f'level_{l}_details', {})
    #     # Sort keys for predictable output
    #     for key in sorted(details.keys()):
    #         val = details[key]
    #         print(f"  {key}_L{l} Detail:\n{val}")
    # print("-" * 30)

    # output_directory_3d = 'haar_lwt_coeffs_3d'
    # save_coefficients_to_files(output_directory_3d, coeffs_3d)
    # print("-" * 30)

    # reconstructed_data_3d = haar_lwt_nd_reconstruct(coeffs_3d)

    # print("\n--- 3D Reconstruction Results ---")
    # print(f"Reconstructed Data:\n{reconstructed_data_3d}")
    # is_reconstruction_accurate_3d = np.allclose(data_3d, reconstructed_data_3d)
    # print(f"Is 3D Reconstruction Accurate? {is_reconstruction_accurate_3d}")
    # if not is_reconstruction_accurate_3d:
    #     print(f"Difference:\n{data_3d - reconstructed_data_3d}")
    # print("-" * 30)


Original Data: [7.343 8.013 8.293 ... 9.517 9.46  9.309]
Decomposition Levels: 3
------------------------------

--- Decomposition Results ---
Final LL (Approximation) Sub-band:
[ 8.66025  10.52225  11.473875  9.89825   8.885375  8.57925   8.941375
  9.44975   9.751125  7.131125  7.585     8.85775   7.9645    7.731375
  7.33875   6.959375  6.54575   7.3785    6.78725   5.662625  5.909375
  5.63275   6.86375   8.18525   9.0265    8.51725   8.152     6.6455
  5.680625  5.14375   4.86425   5.17525   5.9905    6.768375  7.463125
  8.050875  6.94025   7.08775   7.49425   8.512125 11.245875 14.7475
 13.5905   11.668    12.974    14.18025  12.924875 14.74875  17.752
 20.208625 15.42875  10.155875 10.0735   12.075125 10.194375  8.313625
 11.446    12.98575  12.456    12.24025  10.465375  9.359375  9.949375
  9.9925    9.357875  8.31325   7.612625  6.719625  6.17175   5.709625
  6.491875  6.806125  6.165     5.699375  6.435625  7.392125  7.954625
  8.654     8.7615    8.63      8.61975   7.5096